In [2]:
import yaml
import pandas as pd
from pathlib import Path
from config import client
import openai
from IPython.display import display, Markdown

from config import column_names

In [10]:
# load dataframe from csv
df = pd.read_csv('../test_dataframes/jn_items.csv', delimiter='\t', encoding='utf-8')
print(df.head())
print(df.shape)
# load column_names from config.py
df.columns = column_names

dfa = df[['word', 'long_phrase', 'machine_translation']]

# dfa to csv
dfa.to_csv('dfa.csv', sep='\t', encoding='utf-8', index=False)

      WORD|守る|ja  Word  ...  1706274142813_next.jpg  1706274142813.mp3
0      WORD|子|ja  Word  ...  1706274141384_next.jpg  1706274141384.mp3
1    WORD|代わり|ja  Word  ...  1706274138536_next.jpg  1706274138536.mp3
2      WORD|事|ja  Word  ...  1706274135154_next.jpg  1706274135154.mp3
3    WORD|生きる|ja  Word  ...  1706274132200_next.jpg  1706274132200.mp3
4  WORD|タイミング|ja  Word  ...                     NaN  1706274128872.mp3

[5 rows x 24 columns]
(5, 24)


In [25]:
# Example of an OpenAI ChatCompletion request with stream=True and stream_options={"include_usage": True}

# a ChatCompletion request
response = client.chat.completions.create(
    model='gpt-4',
    messages=[
        {'role': 'user', 'content': "Can you give me a summary of the pros and cons of the OpenAI API?"},
    ],
    temperature=0,
    stream=True,
    # stream_options={"include_usage": True}, # retrieving token usage for stream response
)

for chunk in response:
    print(f"choices: {chunk.choices}\nusage: {chunk.usage}")
    # print(f"{chunk.content}")
    print("****************")

choices: [Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)]
usage: None


AttributeError: 'ChatCompletionChunk' object has no attribute 'content'

In [43]:
from openai import OpenAI
client = OpenAI()

stream = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'user', 'content': "Moritz is a great guy from Berlin"},
        # {'role': 'assistant', 'content': "I don't know that"},
        {'role': 'user', 'content': "who is Moritz?"}
    ],
    temperature=0,
    stream=True,
    stream_options={"include_usage": True}, # retrieving token usage for stream response
)

# print the content of the response

# for chunk in stream:
#     if chunk.choices[0].delta.content is not None:
#         print(chunk.choices[0].delta.content, end="")

for chunk in stream:
    print(chunk)



ChatCompletionChunk(id='chatcmpl-9VIXMqyH2Vcb1bayBfZID39Crmnx7', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717246452, model='gpt-4o-2024-05-13', object='chat.completion.chunk', system_fingerprint='fp_319be4768e', usage=None)
ChatCompletionChunk(id='chatcmpl-9VIXMqyH2Vcb1bayBfZID39Crmnx7', choices=[Choice(delta=ChoiceDelta(content='Mor', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717246452, model='gpt-4o-2024-05-13', object='chat.completion.chunk', system_fingerprint='fp_319be4768e', usage=None)
ChatCompletionChunk(id='chatcmpl-9VIXMqyH2Vcb1bayBfZID39Crmnx7', choices=[Choice(delta=ChoiceDelta(content='itz', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717246452, model='gpt-4o-2024-05-13', object='chat.completion.chunk', system_fingerprint='fp_319be4768e', 

In [23]:
from openai import OpenAI

client = OpenAI()

# List of messages to be sent to the chat completion endpoint
messages = [
    {"role": "user", "content": "Always answer briefly. My name is Moritz! What is your name?"},
    {"role": "user", "content": "Guess where I likely come from, based on my name"},
    {"role": "user", "content": "Please sum up this dialogue"},
]

# Create the chat completion stream
stream = client.chat.completions.create(
    model="gpt-4",  # Assuming you're using the latest GPT-4 model
    messages=messages,
    stream=True,
)

# Initialize an empty response container
full_response = ""

# Iterate over the stream and print each chunk
for chunk in stream:
    if 'choices' in chunk:
        delta = chunk['choices'][0]['delta']
        if 'content' in delta:
            full_response += delta['content']
            print(delta['content'], end="")

# Optionally, print the full response at the end
print("\n\nFull Response:\n", full_response)




Full Response:
 


In [15]:
from openai import OpenAI
import os
import nest_asyncio
import asyncio

client = OpenAI()
# Apply the nest_asyncio patch
nest_asyncio.apply()

# Initialize the OpenAI client with your API key


async def main():
    # Create a streaming compion
    response = await client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": "Always answer briefly. My name is Moritz! What is your name?"},
            {"role": "user", "content": "Guess where I likely come from, based on my name"},
            {"role": "user", "content": "Please sum up this dialogue"}
        ],
        stream=True
    )

    # Print the streaming response
    async for chunk in response:
        if 'choices' in chunk:
            choice = chunk['choices'][0]
            if 'delta' in choice and 'content' in choice['delta']:
                print(choice['delta']['content'], end="")

# Run the main function
asyncio.run(main())


TypeError: object Stream can't be used in 'await' expression

In [3]:
csv_file_path = Path("test_dataframes/italian_items/items_long.csv")

    # Load the csv file
df = pd.read_csv(csv_file_path, delimiter='\t')

with open('config/column_names.yaml', 'r') as file:
        data = yaml.safe_load(file)
        column_names = data['column_names']


df.columns = column_names

# select the columns word and long_sentence

df = df[['word', 'long_phrase']]
df.to_csv('test_dataframes/italian_items/items_long_changed.csv', sep='\t', index=False)



In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

# Composition

In [2]:
class Animal:
    def __init__(self):
        self.animal_type = "Generic Animal"

    def make_sound(self):
        print(f"{self.animal_type} sound")

class Dog:
    def __init__(self):
        self.animal_sound = "Woof!"
        self.animal = Animal()  # Create an Animal instance

    def make_sound(self):
        self.animal.make_sound()  # Call base class method directly
        print(self.animal_sound)

# Create objects and call methods
animal = Animal()
dog = Dog()

animal.make_sound()  # Output: Generic Animal sound
dog.make_sound()  # Output: Generic Animal sound
                #        Woof!


Generic Animal sound
Generic Animal sound
Woof!


In [1]:
class Animal:
    def __init__(self):
        self.animal_type = "Generic Animal"

    def make_sound(self):
        print(f"{self.animal_type} sound")

class Dog(Animal):
    def make_sound(self):
        self.animal_type = "Dog"  # Modify attribute before calling base class method
        super().make_sound()
        print("Woof!")

# Create objects and call methods
animal = Animal()
dog = Dog()

animal.make_sound()  # Output: Generic Animal sound
dog.make_sound()  # Output: Dog sound
                #        Woof!


Generic Animal sound
Dog sound
Woof!


In [7]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Markdown Table Example</title>
    <script src="https://cdn.jsdelivr.net/npm/marked@4.0.12/marked.min.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 40em;
        }
        table {
            width: 50%;
            border-collapse: collapse;
        }
        th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>
    <div id="markdown-content"></div>

    <script>
        const markdownTable = `
            | Name   | Age | Occupation   |
            |--------|-----|--------------|
            | John   | 25  | Engineer     |
            | Alice  | 30  | Designer     |
            | Bob    | 22  | Developer    |
            `;

        document.getElementById('markdown-content').innerHTML = marked.parse(markdownTable);
    </script>
</body>
</html>


SyntaxError: invalid decimal literal (3130921398.py, line 18)